# Netflix TV Show Cancellation Analysis
This notebook includes our data wrangling process. Use the Outline/Table of Contents to easily navigate this notebook.

## Step 1: Data Loading and Inspection
First, we focus on loading and inspecting the datasets that allow for analyze Netflix TV show cancellations in 2023. The `data` folder contains three datasets from two data sources. Our third data source, the Reddit API, will be used later in this notebook.
1. Web Scraping: Cancelled TV Shows List  
https://www.kaggle.com/code/natthaburaraksakiet/web-scraping-cancelled-tv-shows-list _Accessed: 30 January 2025_
   - The file `cancelled_netflix_shows.csv` provides information about canceled Netflix TV shows, including their `Year`,`Title`, `Network`, and `Genre`. It was generated by running the script `scraper.py`, a modified version of the script downloaded from Kaggle. The downloaded script was modified only to scrape only Netflix shows rather than all shows. 
2. Netflix: What We Watched (Bi-Annual Engagement Reports: Hours Viewed for Film and TV)  
https://about.netflix.com/en/news/what-we-watched-a-netflix-engagement-report _Accessed: 25 January 2025_: `What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.xlsx`  
https://about.netflix.com/en/news/what-we-watched-the-second-half-of-2023 _Accessed: 25 January 2025_: `What_We_Watched_A_Netflix_Engagement_Report_2023Jul-Dec.xlsx`

### Step 1.0: Install, Import, and Set Options

In [1]:
%pip install pandas
%pip install numpy
%pip install openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For array operations
from datetime import datetime  # For handling date columns

pd.set_option("display.max_rows", 20)  # Limit the number of rows displayed when printing a DataFrame
pd.set_option('display.max_colwidth', None)  # Prevent truncation of long text
pd.set_option('display.width', 200)  # Adjust overall display width to prevent line breaks

### Step 1.1: Load Datasets
#### Load the Canceled Netflix TV Shows dataset

In [3]:
# Load the cancelled Netflix TV shows dataset 
cancelled_netflix_shows_path = '../data/cancelled_netflix_shows.csv'
cancelled_netflix_shows = pd.read_csv(cancelled_netflix_shows_path)

#### Load the Netflix Engagement Reports

In [4]:
# Load the Netflix engagement reports for Jan-Jun and Jul-Dec 2023
engagement_1H2023_path = '../data/What_We_Watched_A_Netflix_Engagement_Report_2023Jan-Jun.xlsx'
engagement_2H2023_path = '../data/What_We_Watched_A_Netflix_Engagement_Report_2023Jul-Dec.xlsx'

# Define which columns to load from the reports 
cols = "B:E"  # Columns B to E include the relevant data: Title, Release Date, Hours Viewed

# Load the Jan-Jun 2023 report; this report contains a combined sheet for TV and films, so 
# we will exclude films later
engagement_1H2023 = pd.read_excel(
    engagement_1H2023_path, 
    # No separate sheets for TV and movies here!
    header=5,  # Skip the first 5 rows, as they contain metadata we do not need
    usecols=cols  # Load only the relevant columns
)

# Load the Jul-Dec 2023 report; this report has a separate "TV" tab, so we can load it 
# directly without needing to filter for TV content
engagement_2H2023 = pd.read_excel(
    engagement_2H2023_path, 
    sheet_name='TV',  # Specify the "TV" tab to exclude film data
    header=5,  # Skip the first 5 rows, as they contain metadata we do not need
    usecols=cols  # Load only the relevant columns
)

### Step 1.2: Inspect Datasets
#### Inspect the Cancelled Netflix TV Shows dataset

In [5]:
print("Canceled Netflix TV Shows DataFrame (pretty-printed):")
display(cancelled_netflix_shows)

# Check the structure of the Cancelled Netflix TV Shows dataset
print("Cancelled Netflix TV Shows dataset info:\n")
print(cancelled_netflix_shows.info(memory_usage=False))

# Check for missing values in the Cancelled Netflix TV Shows dataset:
print("\nMissing values in Cancelled Netflix TV Shows dataset:")
print(cancelled_netflix_shows.isnull().sum())

Canceled Netflix TV Shows DataFrame (pretty-printed):


,Title,Year,Network,Genre
0,Girls5eva,2021 - 2024,Netflix,Comedy
1,Unstable,2023 - 2024,Netflix,Comedy
2,KAOS,2024,Netflix,Drama / Fantasy
3,That '90s Show,2023 - 2024,Netflix,Comedy
4,Buying London,2024,Netflix,Reality
...,...,...,...,...
259,The Politician,2019 - 2020,Netflix,Drama / Comedy
260,Dancing Queen,2018,Netflix,Reality
261,"Boo, Bitch",2022,Netflix,Comedy
262,The Order,2019 - 2020,Netflix,Drama / Horror


Cancelled Netflix TV Shows dataset info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    264 non-null    object
 1   Year     264 non-null    object
 2   Network  264 non-null    object
 3   Genre    263 non-null    object
dtypes: object(4)None

Missing values in Cancelled Netflix TV Shows dataset:
Title      0
Year       0
Network    0
Genre      1
dtype: int64


##### Observations for the Cancelled Netflix TV Shows dataset
1. Data Types:
    - All columns are of type `object`, including the `Year` column, which represents the runtime of the show. This column will need to be processed further to:
        - Distinguish between single-year and multi-year entries (e.g., "2023" vs. "2021-2023")
        - Extract the last year for analysis purposes (to identify shows canceled in 2023)

2. Missing Data:
    - There is 1 missing value in the `Genre` column. Since `Genre` is not critical for this specific project (our focus is on `Title` and `Year`), we can safely drop this column without it affecting our analysis. 

3. Data Quality: 
    - The `Title` column looks complete and will be crucial for merging this dataset with the Netflix reports datasets. 

#### Inspect Netflix Engagement Report H1: Jan-Jun 2023

In [6]:
print("Netflix Engagement Report H1 2023 (pretty-printed):\n")
display(engagement_1H2023)

# Check the structure of the Jan-Jun 2023 report
print("Netflix Engagement Report H1 2023 Info:\n")
print(engagement_1H2023.info(memory_usage=False))

# Check for missing values in the Jan-Jun 2023 report
print("\nMissing Values in Netflix Engagement Report H1 2023:")
print(engagement_1H2023.isnull().sum())

Netflix Engagement Report H1 2023 (pretty-printed):



,Title,Available Globally?,Release Date,Hours Viewed
0,The Night Agent: Season 1,Yes,2023-03-23,812100000
1,Ginny & Georgia: Season 2,Yes,2023-01-05,665100000
2,The Glory: Season 1 // 더 글로리: 시즌 1,Yes,2022-12-30,622800000
3,Wednesday: Season 1,Yes,2022-11-23,507700000
4,Queen Charlotte: A Bridgerton Story,Yes,2023-05-04,503000000
...,...,...,...,...
18209,راس السنة,No,NaT,100000
18210,心が叫びたがってるんだ。,No,NaT,100000
18211,두근두근 내 인생,No,NaT,100000
18212,라디오 스타,No,NaT,100000


Netflix Engagement Report H1 2023 Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18214 entries, 0 to 18213
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Title                18214 non-null  object        
 1   Available Globally?  18214 non-null  object        
 2   Release Date         4855 non-null   datetime64[ns]
 3   Hours Viewed         18214 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)None

Missing Values in Netflix Engagement Report H1 2023:
Title                      0
Available Globally?        0
Release Date           13359
Hours Viewed               0
dtype: int64


##### Observations for Netflix Engagement Report H1: Jan-Jun 2023
1. Missing `Release Date`:
    - The missing values in `Release Date` may indicate that some titles lack specific release information.

2. TV vs. Movies:
    - The dataset includes both TV shows and movies, which will require an inner join with the 'TV' sheet in the Jul-Dec 2023 report to retain only TV shows.

#### Inspect Netflix Engagement Report H2: Jul-Dec 2023

In [7]:
print("Netflix Engagement Report H2 2023 (pretty-printed):")
display(engagement_2H2023)

# Check the structure for the Jul-Dec 2023 report
print("Netflix Engagement Report H2 2023 Info:\n")
print(engagement_2H2023.info(memory_usage=False))

# Check for missing values in the Jul-Dec 2023 report
print("\nMissing Values in Netflix Engagement Report H2 2023:")
print(engagement_2H2023.isnull().sum())

Netflix Engagement Report H2 2023 (pretty-printed):


,Title,Available Globally?,Release Date,Hours Viewed
0,ONE PIECE: Season 1,Yes,2023-08-31,541900000
1,Dear Child: Limited Series // Liebes Kind: Miniserie,Yes,2023-09-07,252800000
2,Who is Erin Carter?: Limited Series,Yes,2023-08-24,286200000
3,Lupin: Part 3,Yes,2023-10-05,274300000
4,The Witcher: Season 3,Yes,2023-06-29,363800000
...,...,...,...,...
6594,We Are Black and British: Season 1,No,NaN,100000
6595,Whitney Cummings: Can I Touch It?,Yes,2019-07-30,100000
6596,Whitney Cummings: Jokes,No,2022-07-26,100000
6597,"Whose Vote Counts, Explained: Limited Series",Yes,2020-09-28,100000


Netflix Engagement Report H2 2023 Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Title                6599 non-null   object
 1   Available Globally?  6599 non-null   object
 2   Release Date         3312 non-null   object
 3   Hours Viewed         6599 non-null   int64 
dtypes: int64(1), object(3)None

Missing Values in Netflix Engagement Report H2 2023:
Title                     0
Available Globally?       0
Release Date           3287
Hours Viewed              0
dtype: int64


##### Observations for Netflix Engagement Report H2: Jul-Dec 2023
1. Missing `Release Date`:
    -  Similar to the Jan-Jun dataset.

2. No Filtering for TV Shows:
    - Because this dataset is already limited to TV shows, no additional filtering by title type is needed.

---

## Step 2: Data Preparation
In Step 1, we loaded and inspected our datasets. Now, we perform data preparation, including normalization, filtering, and merging tasks that will create a cleaned dataset for analysis.

### Step 2.1: Merge Netflix Engagement Reports
To ensure we have complete data for Netflix TV shows in 2023, we merge the Jan-Jun engagement report (in which TV shows and movies combined in a single sheet) with the Jul-Dec engagement report's 'TV' sheet. Only titles that appear in <b>both reports</b> will be retained. This ensures that each title is a TV show for which full-year 2023 data is available. 

In [8]:
# Retain only relevant columns: extract the 'Title' and 'Hours Viewed' 
# columns from each report for simplicity:
engagement_1H2023_trimmed = engagement_1H2023[['Title', 'Hours Viewed']]
engagement_2H2023_trimmed = engagement_2H2023[['Title', 'Hours Viewed']]

# Merge the two reports on 'Title' (inner join to retain only matching titles)
merged_engagement = pd.merge(
    engagement_1H2023_trimmed,
    engagement_2H2023_trimmed, 
    on='Title',  # Join on the 'Title' column
    suffixes=('_1H2023', '_2H2023')  # Add suffixes to distinguish columns
)

# Sum the hours viewed across both halves of the year for each title
merged_engagement['Total Hours Viewed in 2023'] = (
    merged_engagement['Hours Viewed_1H2023'] + merged_engagement['Hours Viewed_2H2023']
)

# Keep only the 'Title' and 'Total Hours Viewed in 2023' columns for further analysis
merged_engagement = merged_engagement[['Title', 'Total Hours Viewed in 2023']]

# Inspect the final merged dataset
print("Total hours viewed in 2023 of Netflix TV shows:")
merged_engagement

Total hours viewed in 2023 of Netflix TV shows:


,Title,Total Hours Viewed in 2023
0,The Night Agent: Season 1,967600000
1,Ginny & Georgia: Season 2,731300000
2,The Glory: Season 1 // 더 글로리: 시즌 1,689700000
3,Wednesday: Season 1,670400000
4,Queen Charlotte: A Bridgerton Story,580600000
...,...,...
5649,Whitney Cummings: Can I Touch It?,200000
5650,Whitney Cummings: Jokes,200000
5651,"Whose Vote Counts, Explained: Limited Series",200000
5652,Yellow Muzi & Friends: Season 1 // 내 마음은 무지,500000


### Step 2.2: Distinguish Multi-Season Shows from Standalone Shows
In this step, we clean the merged engagement report to retain only:
1. <b>Single-season shows:</b> Titles that only appear once in the dataset (no duplicates).
2. <b>The last season for multi-season shows:</b> Titles that have entries for multiple seasons.

We do this by:
- Identifying standalone shows (no ": Season", ": Series", etc. in the title) and retaining them as-is.
- Identifying multi-season shows:
    - Extracting the season number using a regular expression (regex).
    - Keeping only the entry for the last season.
- Removing season information from titles to retain only the show name.

#### Identify standalone shows (no ": Season", ": Series", etc. in the title)
A standalone show is a show that *does not* explicitly mention a season, series, or volume etc. in its title. E.g., "Queen Charlotte: A Bridgerton Story" is a standalone show.

In [9]:
# Create a Boolean mask for titles that do NOT contain ': Season [number]', or ': Series [number]' etc.
standalone_mask = ~merged_engagement['Title'].str.contains(
    r": (?:Season|Series|Volume|Book|Part|Chapter|Temporada|Collection) \d+", 
    regex=True, 
    na=False
)

# Separate standalone shows
standalone_shows = merged_engagement[standalone_mask]

# Print intermediate results to verify correctness
print(f"Number of standalone shows: {standalone_shows.shape[0]}")
print("\nSample of standalone shows:")
standalone_shows


Number of standalone shows: 1112

Sample of standalone shows:


,Title,Total Hours Viewed in 2023
4,Queen Charlotte: A Bridgerton Story,580600000
11,Kaleidoscope: Limited Series,274800000
14,Crash Course in Romance: Limited Series // 일타 스캔들: 리미티드 시리즈,265900000
21,Doctor Cha: Limited Series // 닥터 차정숙: 리미티드 시리즈,253900000
31,Chiquititas (2013),279300000
...,...,...
5648,W. Kamau Bell: Private School Negro,200000
5649,Whitney Cummings: Can I Touch It?,200000
5650,Whitney Cummings: Jokes,200000
5651,"Whose Vote Counts, Explained: Limited Series",200000


#### Extract season numbers for multi-season shows
A multi-season show is a show that *does* explicitly mention a season/series/volume, etc. in its title. E.g., "Wednesday: Season 1" is a multi-season show.

In [10]:
# Create a Boolean mask for multi-season shows (opposite of the standalone mask)
multi_season_mask = ~standalone_mask
multi_season_shows = merged_engagement[multi_season_mask].copy()

# Extract the number from titles using regex: ': Season [number]', or ': Series [number]' etc.
# and store it in a new column named 'Season'
multi_season_shows['Season'] = (
    multi_season_shows['Title']
    .str.extract(
        r": (?:Season|Series|Volume|Book|Part|Chapter|Temporada|Collection) (\d+)", 
        expand=False
    )  # Extracts the season number
    .astype(int)  # Converts the extracted number to an integer for sorting
)

# Print intermediate results to verify correctness
print(f"Total number of seasons of multi-season shows: {multi_season_shows.shape[0]}")
print("\nSample of multi-season shows with extracted seasons:")
multi_season_shows


Total number of seasons of multi-season shows: 4542

Sample of multi-season shows with extracted seasons:


,Title,Total Hours Viewed in 2023,Season
0,The Night Agent: Season 1,967600000,1
1,Ginny & Georgia: Season 2,731300000,2
2,The Glory: Season 1 // 더 글로리: 시즌 1,689700000,1
3,Wednesday: Season 1,670400000,1
5,You: Season 4,471400000,4
...,...,...,...
5642,Two Santas: Season 1 // Zwei Weihnachtsmänner: Staffel 1,900000,1
5643,Undercover Food Fighters: Season 1 // 위장취업: 시즌 1,1000000,1
5645,Vem Dançar com o Universo Z: Season 3,200000,3
5646,Vietnamese Horror Story: Season 1 // Chuyện ma gần nhà: Mùa 1,200000,1


#### Retain only the last season etc. for multi-season shows

In [11]:
# Create a "Show Name" column by removing ": Season #", or ": Series #" etc. and everything 
# that comes after, ensuring that all rows for the same show are grouped correctly
multi_season_shows['Show_Name'] = multi_season_shows['Title'].str.replace(
    r": (Season|Series|Volume|Book|Part|Chapter|Temporada|Collection) \d+.*", 
    "", 
    regex=True
).str.strip()

# Normalize "Show_Name" for consistent grouping by converting to lowercase and removing special characters
multi_season_shows['Normalized_Show_Name'] = (
    multi_season_shows['Show_Name']
    .str.lower()
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
    .str.strip()  # Remove leading/trailing whitespace
)

# Group by "Normalized_Show_Name" and find the row with the highest season
last_season_shows = (
    multi_season_shows.loc[multi_season_shows.groupby('Normalized_Show_Name')['Season'].idxmax()]
)

# Print intermediate results to verify correctness, sorting by the highest season for clarity
print(f"Number of shows after retaining last season entries: {last_season_shows.shape[0]}")
print("\nSample of last season shows sorted by highest season (descending):")
display(last_season_shows.sort_values(by='Season', ascending=False)[['Title', 'Season', 'Total Hours Viewed in 2023']].head(10))


Number of shows after retaining last season entries: 2352

Sample of last season shows sorted by highest season (descending):


,Title,Season,Total Hours Viewed in 2023
915,Survivor (2000): Season 32: Kaôh Rōng,32,23100000
4690,Top Gear (2003): Season 31,31,1000000
2623,The Real World: Season 28,28,4400000
2318,Thomas and Friends: Season 24,24,8700000
2386,Hell's Kitchen (2005): Season 21,21,5600000
3124,Naruto Shippuden: Season 21 // NARUTO-ナルト- 疾風伝: 外伝編,21,4500000
5042,Bob the Builder (1999): Season 21,21,800000
2966,Intervention: Season 21,21,3500000
4944,Made in Chelsea: Season 19,19,900000
1517,Grey's Anatomy: Season 19,19,41300000


#### Concatenate resulting DataFrames
Finally, we concatenate the two resulting separate datasets to obtain a single DataFrame that contains the stand-alone shows and only the last season of multi-season shows.

In [12]:
# Combine the two datasets using the concatenate function
final_engagement_report = pd.concat([standalone_shows, last_season_shows], ignore_index=True)

print(f"Total number of shows in the final engagement report: {final_engagement_report.shape[0]}")

# Print a sample of the combined dataset
print("\nSample of the final engagement report:")
display(final_engagement_report[['Title', 'Total Hours Viewed in 2023']].head(10))

Total number of shows in the final engagement report: 3464

Sample of the final engagement report:


,Title,Total Hours Viewed in 2023
0,Queen Charlotte: A Bridgerton Story,580600000
1,Kaleidoscope: Limited Series,274800000
2,Crash Course in Romance: Limited Series // 일타 스캔들: 리미티드 시리즈,265900000
3,Doctor Cha: Limited Series // 닥터 차정숙: 리미티드 시리즈,253900000
4,Chiquititas (2013),279300000
5,The Good Bad Mother: Limited Series // 나쁜엄마: 리미티드 시리즈,208000000
6,Stranger Things 4,230700000
7,MH370: The Plane That Disappeared: Limited Series,133300000
8,Muted: Limited Series // El silencio: Miniserie,139800000
9,The Interest of Love: Limited Series // 사랑의 이해: 리미티드 시리즈,115200000


### Step 2.3: Filter Relevant TV Shows from the Cancelled Netflix TV Shows Dataset
In this step, we filter the canceled Netflix TV shows dataset to include only the shows relevant to our 2023 analysis. Using the `Year` column, we retain:
1. <b>Shows that only ran in 2023:</b> These are identified by `Year = "2023"`.
2. <b>Shows that ended in 2023 (i.e., cancelled):</b> (`Year` in the format `year - 2023`).

This ensures we exclude:
- Shows that started in 2023 but ended later (e.g., `2023 - 2025`).
- Any other entries outside these two scenarios.

#### Define regex patterns for relevant scenarios

In [13]:
# Case 1: Shows that only ran in 2023. Matches rows where the value in the Year column is exactly "2023" 
# (shows that only ran in 2023)
single_year_pattern = r"^2023$"

# Case 2: Shows that ended in 2023. Matches rows where the value in the Year column ends with "- 2023" 
# (shows that ended in 2023 after running for multiple years), and due to the nature of the dataset we 
# know that ended == cancelled.
ended_in_2023_pattern = r"^\d{4} - 2023$"  

#### Apply filters to the cancelled Netflix TV shows dataset

In [14]:
# Retain only shows matching either case 1 or case 2:
filtered_cancelled_netflix_shows = cancelled_netflix_shows[
    cancelled_netflix_shows['Year'].str.match(single_year_pattern) |  # Case 1
    cancelled_netflix_shows['Year'].str.match(ended_in_2023_pattern)  # Case 2
]

# Drop the Genre column since it is not needed for our analysis
filtered_cancelled_shows = filtered_cancelled_netflix_shows.drop(columns=['Genre'])

# Print the total number of relevant canceled TV shows
print(f"Total number of relevant canceled Netflix TV shows: {filtered_cancelled_shows.shape[0]}")

# Print the cleaned DataFrame
print("\nFiltered Canceled Netflix Shows (Relevant to 2023):")
display(filtered_cancelled_shows)

Total number of relevant canceled Netflix TV shows: 23

Filtered Canceled Netflix Shows (Relevant to 2023):


,Title,Year,Network
8,My Dad the Bounty Hunter,2023,Netflix
9,"Ada Twist, Scientist",2021 - 2023,Netflix
10,Everything Now,2023,Netflix
11,Emergency: NYC,2023,Netflix
12,Dance 100,2023,Netflix
...,...,...,...
54,Lockwood & Co.,2023,Netflix
55,Ridley Jones,2021 - 2023,Netflix
58,Bling Empire: New York,2023,Netflix
60,Freeridge,2023,Netflix


### Step 2.4: Merge Cancelled Netflix Shows with Engagement Data

In [15]:
# Normalize "Title" for matching purposes: convert to lowercase and remove special characters
filtered_cancelled_shows['Normalized_Show_Name'] = (
    filtered_cancelled_shows['Title']
    .str.lower()
    .str.replace(r"[^\w\s]", "", regex=True)  # Remove special characters
    .str.strip()  # Remove leading/trailing whitespace
)

# Clean the engagement report: drop rows with missing show names to avoid NaN values messing up our matching logic
final_engagement_report_cleaned = final_engagement_report.dropna(subset=['Normalized_Show_Name']).copy()

# For each cancelled show title, note the matching show in the engagement report, if any
final_engagement_report_cleaned['Cancelled_Show_Name'] = np.where(
    final_engagement_report_cleaned['Normalized_Show_Name'].isin(filtered_cancelled_shows['Normalized_Show_Name'].values),
    final_engagement_report_cleaned['Normalized_Show_Name'],
    None
)

In the cleaned final engagement report DataFrame above, some rows have a 'Cancelled_Show_Name', whereas others do not.

We now create two separate DataFrames:
1. One for the cancelled shows (where 'Cancelled_Show_Name' is not None).
2. One for the non-cancelled shows (where 'Cancelled_Show_Name' is None).

#### Create DataFrame for Cancelled Shows

In [16]:
merged_cancelled = final_engagement_report_cleaned.dropna(subset=['Cancelled_Show_Name'])

# We only need the 'Title', 'Normalized_Show_Name', and 'Total Hours Viewed in 2023' columns.
merged_cancelled = merged_cancelled[['Title', 'Normalized_Show_Name', 'Total Hours Viewed in 2023']]

# Reset the index for a cleaner output
merged_cancelled = merged_cancelled.reset_index(drop=True)

# Print the merged DataFrame
print("Merged cancelled shows with viewing hours:")
merged_cancelled

Merged cancelled shows with viewing hours:


,Title,Normalized_Show_Name,Total Hours Viewed in 2023
0,"Ada Twist, Scientist: Season 4",ada twist scientist,7700000
1,Agent Elvis: Season 1,agent elvis,6600000
2,Bake Squad: Season 2,bake squad,57600000
3,Bling Empire: New York: Season 1,bling empire new york,38700000
4,Copenhagen Cowboy: Season 1,copenhagen cowboy,8900000
5,Dance 100: Season 1,dance 100,18800000
6,Emergency: NYC: Season 1,emergency nyc,48100000
7,Freeridge: Season 1,freeridge,26700000
8,Glamorous: Season 1,glamorous,87900000
9,Invisible City: Season 2 // Cidade Invisível: Temporada 2,invisible city,33900000


We observe that only 20 out of the 23 cancelled netflix TV shows were matched, and so we check which shows are missing.

In [17]:
# Find shows from the cancelled dataset that weren't located in the engagement dataset
unmatched_cancelled_shows = filtered_cancelled_shows[
    ~filtered_cancelled_shows['Normalized_Show_Name'].isin(merged_cancelled['Normalized_Show_Name'])
]

# Print the unmatched shows: cancelled shows that were not matched in the merged engagement report dataset
print("Cancelled shows not found in engagement reports:")
unmatched_cancelled_shows

Cancelled shows not found in engagement reports:


,Title,Year,Network,Normalized_Show_Name
10,Everything Now,2023,Netflix,everything now
17,Obliterated,2023,Netflix,obliterated
31,Captain Fall,2023,Netflix,captain fall


The above TV shows only appear in Jul-Dec (H2), not in Jan-Jun (H1). Since these shows did not appear in one of the reports, their 2023 hours viewed data is incomplete, and thus, they should be dropped. 

#### Create DataFrame for Non-Cancelled Shows
Now that we know we matched 20 cancelled shows, we randomly select 20 non-cancelled shows from the merged engagement reports to compare the cancelled shows with.

In [18]:
# Filter the engagement report to get only the non-cancelled shows.
non_cancelled_shows = final_engagement_report_cleaned[final_engagement_report_cleaned['Cancelled_Show_Name'].isna()]

# Randomly select 20 shows from this filtered DataFrame
random_non_cancelled_shows = non_cancelled_shows.sample(n=20, random_state=26)  # Using random_state for reproducibility

# We only need the same three columns:
random_non_cancelled_shows = random_non_cancelled_shows[['Title', 'Normalized_Show_Name', 'Total Hours Viewed in 2023']]

# Reset the index for a cleaner output
random_non_cancelled_shows = random_non_cancelled_shows.reset_index(drop=True)

# Print the merged DataFrame
print("Randomly selected non-cancelled shows for comparison:")
random_non_cancelled_shows

Randomly selected non-cancelled shows for comparison:


,Title,Normalized_Show_Name,Total Hours Viewed in 2023
0,Record of Youth: Season 1 // 청춘기록: 시즌 1,record of youth,45400000
1,The Guest: Season 1 // 손 the guest: 시즌1,the guest,10000000
2,Dated and Related: Season 1,dated and related,13700000
3,Warrior Nun: Season 2,warrior nun,33500000
4,Exposed: The Case Of Keli Lane: Season 1,exposed the case of keli lane,400000
5,The Beautiful Lie: Season 1,the beautiful lie,600000
6,Extraordinary Attorney Woo: Season 1 // 이상한 변호사 우영우: 시즌 1,extraordinary attorney woo,204800000
7,Pending Train: Season 1 // ペンディングトレイン―8時23分、明日 君と: シーズン1,pending train,12600000
8,Come Dine with Me: Series 13,come dine with me,5400000
9,Attack on Pearl Harbor: Minute by Minute: Season 1,attack on pearl harbor minute by minute,4000000


#### Exporting the outputs
The two seperate tables are exported to .csv files that we will use for visualizations and further analysis later.

In [19]:
# Exporting the final output for cancelled shows:
merged_cancelled.to_csv("../output/cancelled_shows_hours_viewed.csv")

# Exporting the final output for non-cancelled shows:
random_non_cancelled_shows.to_csv("../output/noncancelled_shows_hours_viewed.csv")

---

## Step 3: Reddit API

### Step 3.0: Install Required Libraries and Set Up the Reddit API Connection
Before using the Reddit API, we need to install the `praw` library. It simplifies authentication and API interactions. To create and load a .env file for API credentials and prevent others from accessing API keys, install the `python-dotenv` library.

In [20]:
%pip install praw
%pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import praw
import os

#### Connecting to the API

In [22]:
# Retrieve credentials
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
USER_AGENT = os.getenv("USER_AGENT")

# Authenticate using Reddit credentials
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
)

# Test connection
print("Reddit API is read-only:", reddit.read_only)

Reddit API is read-only: True


### Step 3.1: Test Subreddit Access
Verify that we can access the two subreddits "r/NetflixBestOf" and "r/television" using the Reddit API.

In [23]:
subreddits_to_search = ['NetflixBestOf', 'television']

# Test access to the subreddits
for subreddit_name in subreddits_to_search:
    try:
        subreddit = reddit.subreddit(subreddit_name)  # Access subreddit
        
        print(f"Successfully accessed subreddit: {subreddit.display_name}")
    except Exception as e:
        print(f"Error accessing subreddit {subreddit_name}: {e}")

Successfully accessed subreddit: NetflixBestOf
Successfully accessed subreddit: television


### Step 3.2: Define a Function to Search for Show Mentions in 2023
This function will search for posts mentioning a show in the specified subreddits and return the total number of posts and comments (i.e., <b>discussion volume</b>).

In [24]:
def fetch_2023_discussion(subreddit_name: str, show_title: str) -> tuple[int, int]:
    """
    Fetches 2023 Reddit discussion volume: returns the total number of 
    posts and comments found for the given show in the subreddit.
    """
    total_posts = 0  # Counter to track the number of posts mentioning the show
    total_comments = 0  # Counter to track the total number of comments on those posts

    # Define the time range for the search (January 1, 2023 - December 31, 2023)
    start_timestamp = int(datetime(2023, 1, 1).timestamp())
    end_timestamp = int(datetime(2024, 1, 1).timestamp())  # Start of 2024 (exclusive boundary)

    # Access the specified subreddit
    subreddit = reddit.subreddit(subreddit_name)

    # Search for posts mentioning the show title, sorting by newest first, within the last year (2023)
    # The `limit=500` parameter ensures we do not fetch more than 500 posts per query
    # We chose 500 as a reasonable balance between performance and capturing enough discussions
    # A smaller limit (e.g., 10-20) might miss out on relevant discussions, while a larger limit (e.g., 800-1000)
    # could lead to excessive API calls and slow performance without significantly improving insights.
    for post in subreddit.search(query=show_title, sort="new", time_filter="all", limit=500):
        # Check if the post falls within the desired 2023 time range!

        if start_timestamp <= post.created_utc < end_timestamp:
            total_posts += 1  # Increment the post counter

            # Load all comments (ignoring 'More Comments' placeholders) to count accurately
            post.comments.replace_more(limit=0)
            total_comments += len(post.comments.list())  # Add the number of comments in the post

    return total_posts, total_comments

In [25]:
def get_discussion_counts(show):
    """
    Helper function to sum discussion counts across subreddits. Given a show title, this 
    function retrieves the total number of Reddit posts and comments across all subreddits 
    in the `subreddits_to_search` list.
    """
    total_posts, total_comments = 0, 0 # Initialize counters for posts and comments

    for sub in subreddits_to_search:
        posts, comments = fetch_2023_discussion(sub, show) 
        total_posts += posts
        total_comments += comments

    # Return as a Pandas Series so `apply()` can correctly assign values to multiple columns
    return pd.Series([total_posts, total_comments])

### Step 3.3: Fetch Reddit Data for the 20 Cancelled and 20 Non-Cancelled Shows
We loop through the shows in the cancelled and non-cancelled DataFrames generated in the previous section, and collect the results.

#### Fetching Reddit Data for the Cancelled TV Shows

In [26]:
# Fetch Reddit data for cancelled shows
merged_cancelled[['Total Posts (2023)', 'Total Comments (2023)']] = merged_cancelled['Normalized_Show_Name'].apply(get_discussion_counts)

# Export results to CSV
merged_cancelled[['Title', 'Normalized_Show_Name', 'Total Posts (2023)', 'Total Comments (2023)']].to_csv("../output/cancelled_shows_reddit_2023.csv")

# Print results for quick verification
display(merged_cancelled[['Title', 'Normalized_Show_Name', 'Total Posts (2023)', 'Total Comments (2023)']])

,Title,Normalized_Show_Name,Total Posts (2023),Total Comments (2023)
0,"Ada Twist, Scientist: Season 4",ada twist scientist,1,19
1,Agent Elvis: Season 1,agent elvis,6,629
2,Bake Squad: Season 2,bake squad,0,0
3,Bling Empire: New York: Season 1,bling empire new york,65,4922
4,Copenhagen Cowboy: Season 1,copenhagen cowboy,4,159
5,Dance 100: Season 1,dance 100,1,31
6,Emergency: NYC: Season 1,emergency nyc,0,0
7,Freeridge: Season 1,freeridge,2,483
8,Glamorous: Season 1,glamorous,5,537
9,Invisible City: Season 2 // Cidade Invisível: Temporada 2,invisible city,2,45


#### Fetching Reddit Data for the Non-Cancelled TV Shows

In [27]:
# Fetch Reddit data for non-cancelled shows
random_non_cancelled_shows[['Total Posts (2023)', 'Total Comments (2023)']] = random_non_cancelled_shows['Normalized_Show_Name'].apply(get_discussion_counts)

# Export results to CSV
random_non_cancelled_shows[['Title', 'Normalized_Show_Name', 'Total Posts (2023)', 'Total Comments (2023)']].to_csv("../output/noncancelled_shows_reddit_2023.csv")

# Print results for quick verification
display(random_non_cancelled_shows[['Title', 'Normalized_Show_Name', 'Total Posts (2023)', 'Total Comments (2023)']])

,Title,Normalized_Show_Name,Total Posts (2023),Total Comments (2023)
0,Record of Youth: Season 1 // 청춘기록: 시즌 1,record of youth,0,0
1,The Guest: Season 1 // 손 the guest: 시즌1,the guest,93,5696
2,Dated and Related: Season 1,dated and related,8,311
3,Warrior Nun: Season 2,warrior nun,12,858
4,Exposed: The Case Of Keli Lane: Season 1,exposed the case of keli lane,5,69
5,The Beautiful Lie: Season 1,the beautiful lie,5,309
6,Extraordinary Attorney Woo: Season 1 // 이상한 변호사 우영우: 시즌 1,extraordinary attorney woo,5,115
7,Pending Train: Season 1 // ペンディングトレイン―8時23分、明日 君と: シーズン1,pending train,1,23
8,Come Dine with Me: Series 13,come dine with me,2,252
9,Attack on Pearl Harbor: Minute by Minute: Season 1,attack on pearl harbor minute by minute,45,3856


-----

## Step 4: Aggregated Dataset (Hours Viewed + Reddit API data)

In [28]:
# Add cancellation status to both datasets
merged_cancelled['Cancelled'] = True
random_non_cancelled_shows['Cancelled'] = False

# Merge the datasets
aggregated_df = pd.concat([merged_cancelled, random_non_cancelled_shows], ignore_index=True)

# Export results to CSV
aggregated_df.to_csv('../output/aggregated.csv')

# Print results for quick verification
aggregated_df

,Title,Normalized_Show_Name,Total Hours Viewed in 2023,Total Posts (2023),Total Comments (2023),Cancelled
0,"Ada Twist, Scientist: Season 4",ada twist scientist,7700000,1,19,True
1,Agent Elvis: Season 1,agent elvis,6600000,6,629,True
2,Bake Squad: Season 2,bake squad,57600000,0,0,True
3,Bling Empire: New York: Season 1,bling empire new york,38700000,65,4922,True
4,Copenhagen Cowboy: Season 1,copenhagen cowboy,8900000,4,159,True
...,...,...,...,...,...,...
35,In the Dark (2019): Season 4,in the dark 2019,15000000,5,223,False
36,Terrace House: Aloha State: Part 4 // テラスハウス Aloha State: Part 4,terrace house aloha state,1300000,13,871,False
37,Spirit Riding Free: Riding Academy: Part 2,spirit riding free riding academy,12600000,65,5188,False
38,DRIFTING DRAGONS: Season 1 // 空挺ドラゴンズ: シーズン1,drifting dragons,1700000,0,0,False
